In this notebook we try to estimate the dimensionality of the dynamic trajectory subspace.

### Helpful articles
- [Supplementary material from AI Poincare](https://journals.aps.org/prl/supplemental/10.1103/PhysRevLett.126.180604/poincare_supplemental_materialv2.pdf)
- [Interpretable conservation law estimation by deriving the symmetries of dynamics from trained deep neural networks](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.103.033303) cited in AI Poincare.

In [ ]:
from experiments import *
import autoencoders as ae

In [ ]:
import wandb

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep_k',
    'metric': {
        'goal': 'minimize',
        'name': 'pendulum_1_test_mse'
    },
    'parameters': {
        'k': {'values': [1, 5, 10, 20, 32]}
    }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="AE new (gae sweep)")

In [ ]:
def main():
    wandb.init(project="AE new (gae sweep)")
    k = wandb.config.k
    suite = ae.TrajectoryAutoencoderSuite(Pendulum, criterion=ae.GAELoss(k=k))
    suite.analyze_n_eff()

In [ ]:
wandb.agent(sweep_id, function=main, count=1)